In [1]:
import pandas as pd
import numpy as np

In [2]:
# !pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.0 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
data = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding="ISO-8859-1",names=column_names)

In [4]:
(data["target"] == 4).sum()

800000

In [5]:
train = data[["text", "target"]]
train.head()

,text,target
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [6]:
# # Select the first 10000 rows of 'text' and 'target'
# train_part1 = data[["text", "target"]][:10000]

# # Select rows from 80000 to 90000 of 'text' and 'target'
# train_part2 = data[["text", "target"]][800000:810000]

# # Concatenate both parts into one DataFrame
# train = pd.concat([train_part1, train_part2])
# train

In [7]:
train["target"]= train["target"].replace(4,1)

/tmp/ipykernel_34/1677995025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["target"]= train["target"].replace(4,1)


In [8]:
train["target"].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [9]:
# Storing data in lists.
text, target = list(train['text']), list(train['target'])

In [11]:
import spacy
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import re
import time
# Load spaCy English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Download NLTK data
nltk.download("punkt")
nltk.download("stopwords")
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

# # Tokenization and preprocessing
# def preprocess(doc):
#     # Tokenize using NLTK
#     tokens = word_tokenize(doc)

#     # Remove punctuation and convert to lowercase
#     tokens = [word.lower() for word in tokens if word.isalpha()]

#     # Remove stop words
#     stop_words = set(stopwords.words("english"))
#     filtered_tokens = [word for word in tokens if word not in stop_words]
#     filtered_tokens = [word for word in filtered_tokens if len(word) > 1]

#     # Create a spaCy document for lemmatization
#     spacy_doc = nlp(' '.join(filtered_tokens))

#     # Lemmatization using spaCy
#     lemmatized_tokens = [token.lemma_ for token in spacy_doc]
    
#     return lemmatized_tokens
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# %%time
# train.text = train.text.apply(lambda x: preprocess(x))

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train["text"],train["target"], test_size=1-0.8, random_state=42)
print(X_train.shape)
print(y_train.shape)

(1280000,)
(1280000,)


In [14]:
len(X_test)
# len(y_test)

320000

In [16]:
# from tqdm import tqdm
# tokenized_set= []
# for i in tqdm(range(len(text))):
#     processed = preprocess(text[i])
#     tokenized_set.append(processed)

In [15]:
# df_train.head()

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_words=10000
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_train)
sequence_train=tokenizer.texts_to_sequences(X_train)
sequence_test=tokenizer.texts_to_sequences(X_test)

2024-05-15 17:35:10.742144: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 17:35:10.742271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 17:35:10.876544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [17]:
sequence_train[:10]

[[196323,
  21,
  290,
  38,
  364,
  34,
  4,
  2432,
  1323,
  59742,
  2586,
  120043,
  162,
  17,
  324,
  29,
  18,
  8,
  5,
  862,
  213,
  27,
  729,
  29342],
 [925, 3, 4067, 26, 587, 9, 511, 2, 22, 4, 18, 23, 3, 41754],
 [38278, 13, 2243, 35, 34, 15, 10, 14, 5873],
 [231,
  118,
  76,
  250,
  4,
  1483,
  97,
  1103,
  72,
  115,
  88434,
  421,
  180,
  111,
  146,
  37,
  250,
  45,
  41],
 [120044, 140, 55, 4, 1827, 1296],
 [54,
  87,
  50,
  245,
  5,
  1175,
  946,
  196324,
  120045,
  196325,
  33060,
  51864,
  6350,
  70920,
  394,
  2,
  332,
  245,
  14,
  74],
 [33061, 53, 613, 82, 57, 22, 1521, 2, 161, 196326, 7952, 402],
 [196327,
  162,
  179,
  906,
  44,
  17757,
  23,
  23393,
  295,
  3,
  273,
  30,
  519,
  441,
  120,
  391,
  635,
  2,
  7,
  212],
 [5,
  5150,
  2674,
  388,
  3,
  447,
  2522,
  226,
  1,
  157,
  1059,
  5,
  2948,
  229,
  342,
  1004,
  2522],
 [10119,
  747,
  166,
  302,
  18,
  120,
  721,
  595,
  630,
  9,
  6,
  22,
  1684,

In [18]:
# train_documents = [text.split() for text in df_train.text] 

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
data_train=pad_sequences(sequence_train)
data_train.shape

(1280000, 118)

In [20]:
T=data_train.shape[1]
data_test=pad_sequences(sequence_test,maxlen=T)
data_test.shape

(320000, 118)

In [21]:
data_train[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,   925,     3,  4067,    26,
         587,     9,   511,     2,    22,     4,    18,    23,     3,
       41754], dtype=int32)

In [22]:
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding,Dropout
from tensorflow.keras.models import Model

# Embedimg Dimension 32 , 2 Conv layers

In [23]:
D=32
input_layer=Input((T,))
x=Embedding(len(tokenizer.word_index)+1,D)(input_layer)
# x=Dropout(0.5)(x)
x=Conv1D(64,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
# x=Dropout(0.5)(x)
x=Conv1D(128,3,activation='relu')(x)
x=GlobalMaxPooling1D()(x)
# x=Dropout(0.5)(x)
x=Dense(1,activation='sigmoid')(x)
model=Model(input_layer,x)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 118)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 118, 32)        │    19,035,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 116, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 38, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 36, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,066,209 (72.73 MB)

 Trainable params: 19,066,209 (72.73 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
len(y_train)

1280000

In [25]:
vals,counts=np.unique(y_test,return_counts=True)
counts

array([159494, 160506])

In [26]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])
cnn_senti=model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=1,batch_size=64)

   34/20000 ━━━━━━━━━━━━━━━━━━━━ 1:33 5ms/step - accuracy: 0.5119 - loss: 0.6935

I0000 00:00:1715794592.731878     159 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20000/20000 ━━━━━━━━━━━━━━━━━━━━ 102s 5ms/step - accuracy: 0.7487 - loss: 0.5050 - val_accuracy: 0.7812 - val_loss: 0.4603


In [27]:
data_test.shape

(320000, 118)

In [28]:
y_preds=model.predict(data_test)
y_pred=np.argmax(y_preds,axis=1)
y_pred

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step


array([0, 0, 0, ..., 0, 0, 0])

In [29]:
y_pred = y_preds>0.5

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
# y_pred[:100]

In [32]:
# y_preds[:100]

In [33]:
accuracy_score(y_test,y_pred)

0.7812375

In [34]:
vals,counts=np.unique(y_pred,return_counts=True)

In [35]:
counts

array([149394, 170606])

# Embeding Dimension = 16 , 3 conv layers

In [36]:
D=16
input_layer=Input((T,))
x=Embedding(len(tokenizer.word_index)+1,D)(input_layer)
x=Conv1D(32,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(64,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(128,3,activation='relu')(x)
x=GlobalMaxPooling1D()(x)
x=Dense(1,activation='sigmoid')(x)
model=Model(input_layer,x)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 118)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 118, 16)        │     9,517,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 116, 32)        │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 38, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 36, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 12, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 10, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,550,193 (36.43 MB)

 Trainable params: 9,550,193 (36.43 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])
cnn_senti=model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=1,batch_size=64)

20000/20000 ━━━━━━━━━━━━━━━━━━━━ 77s 4ms/step - accuracy: 0.7444 - loss: 0.5091 - val_accuracy: 0.7776 - val_loss: 0.4665


In [38]:
y_preds=model.predict(data_test)
y_pred=np.array(y_preds>0.5)
# y_pred

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step


In [39]:
from sklearn.metrics import accuracy_score

In [40]:
accuracy_score(y_test,y_pred)

0.777603125

In [41]:
vals,counts=np.unique(y_pred,return_counts=True)

In [42]:
counts

array([163715, 156285])

# Kernel Size 5x5 , Embeding Dimension 32 , 2 Conv Layers

In [43]:
D=32
input_layer=Input((T,))
x=Embedding(len(tokenizer.word_index)+1,D)(input_layer)
x=Conv1D(64,5,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(128,5,activation='relu')(x)
x=GlobalMaxPooling1D()(x)
x=Dense(1,activation='sigmoid')(x)
model=Model(input_layer,x)
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 118)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 118, 32)        │    19,035,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 114, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 38, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 34, 128)        │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,086,689 (72.81 MB)

 Trainable params: 19,086,689 (72.81 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])
cnn_senti=model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=1,batch_size=64)

20000/20000 ━━━━━━━━━━━━━━━━━━━━ 103s 5ms/step - accuracy: 0.7937 - loss: 0.4375 - val_accuracy: 0.8276 - val_loss: 0.3835


In [45]:
y_preds=model.predict(data_test)
y_pred=np.array(y_preds>0.5)
# y_pred

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step


In [46]:
accuracy_score(y_test,y_pred)

0.827603125

In [47]:
vals,counts=np.unique(y_pred,return_counts=True)

In [48]:
counts

array([158409, 161591])

In [86]:
y_test = np.array(y_test)

# Choosing 10 random samples from testset to show results on

In [91]:
idxs = np.random.randint(0,data_test.shape[0],(10),dtype=int)
X_sample = data_test[idxs]
X_sample_data = np.array(X_test)[idxs]
y_sample = y_test[idxs]

In [94]:
X_sample

array([[    0,     0,     0, ...,     0,   131,  4197],
       [    0,     0,     0, ...,  2931, 12517, 21485],
       [    0,     0,     0, ...,     2,   290,   103],
       ...,
       [    0,     0,     0, ...,    41,    25,   783],
       [    0,     0,     0, ...,   148,    40,   348],
       [    0,     0,     0, ...,   127,    13,   281]], dtype=int32)

In [95]:
X_sample_data

array(['getting anxious ',
       'Thinks she might just die from the sweltering heat of her bedroom. Ewwwww hotttt ',
       'Late mornin to ya twitter! ',
       '@GarretOhm we just had our m3 and your friend Dink came up, we were all laughing ',
       'At my photoshoot, waiting for my clients to arrive. Wish there was highspeed internet here ',
       'tonite was fun and full of surprises  tmw is gonna be eh...',
       '@ArmonDeacon Night ',
       'my dread was validated... today was horrible ',
       "but Annie didn't do anything ",
       'im about to hit 100 followers need 7 more come on baby '],
      dtype=object)

In [96]:
y_sample

array([0, 0, 1, 1, 0, 1, 1, 0, 0, 0])

In [98]:
y_preds=model.predict(X_sample)
y_pred=np.array(y_sample>0.5)
y_pred.astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([0, 0, 1, 1, 0, 1, 1, 0, 0, 0])

In [100]:
accuracy_score(y_sample,y_pred)

1.0